In [2]:
from pyrocko import util, model, io, trace, moment_tensor, gmtpy,orthodrome
from pyrocko import orthodrome as od
from pyrocko.client import catalog
from pyrocko.automap import Map
import pyrocko.moment_tensor as pmt
from pyrocko.plot import mpl_color
from pyrocko.guts import load
# from seiscloud import plot as scp
# from seiscloud import cluster as scc
import numpy as np
import os
# import shutil
import matplotlib.pyplot as plt
import pygmt
import grond

In [3]:
workdir='../'
reportdir=os.path.join(workdir,'report')                                #main report dir

catdir=os.path.join(workdir,'CAT')
catname=os.path.join(catdir,'catalogue_flegrei_VLP_gilberto.pf')
refevents=model.load_events(catname)

mttargets = [ev for ev in refevents]
badmtsols = ['']
print('All events in catalogue:', len(mttargets))
goodmttargets = [ev for ev in mttargets if ev.name not in badmtsols]
print('Good events in catalogue:', len(goodmttargets))
grondevs = []

freq=[]
duration=[]
lat=[]
lon=[]
for vrs in ['cmt_LP_oscill_']:
    for ev in goodmttargets:
        targetdir = os.path.join(reportdir, ev.name, vrs + ev.name)
        #if not os.path.isdir(targetdir):
            #print('missing report dir "', vrs, '" for event:',ev.name)
        if os.path.isdir(targetdir):
            fname = os.path.join(targetdir, 'stats.yaml')     # take BEST results
            VLP_stats=load(filename=fname)

            for stat in VLP_stats.parameter_stats_list:
                if stat.name=='duration':
                    duration.append(stat.best)
                if stat.name=='frequency':
                    freq.append(stat.best)

mean_f=np.mean(freq)
std_f=np.std(freq)
print(f'\nVLP frequency: {mean_f:1.4} +- {std_f:1.2}')

mean_d=np.mean(duration)
std_d=np.std(duration)
print(f'VLP duration: {mean_d:2.4} +- {std_d:2.4}')

All events in catalogue: 10
Good events in catalogue: 10

VLP frequency: 0.1143 +- 0.0017
VLP duration: 26.93 +- 4.404
